# Section: Securing Federated Learning

- Lesson 1: Trusted Aggregator
- Lesson 2: Intro to Additive Secret Sharing
- Lesson 3: Intro to Fixed Precision Encoding
- Lesson 4: Secret Sharing + Fixed Precision in PySyft
- Final Project: Federated Learning wtih Encrypted Gradient Aggregation

# Lesson: Federated Learning with a Trusted Aggregator

In the last section, we learned how to train a model on a distributed dataset using Federated Learning. In particular, the last project aggregated gradients directly from one data owner to another. 

However, while in some cases it could be ideal to do this, what would be even better is to be able to choose a neutral third party to perform the aggregation.

As it turns out, we can use the same tools we used previously to accomplish this.

# Project: Federated Learning with a Trusted Aggregator

In [87]:
# try this project here!

In [88]:
import syft as sy
import torch as th
hook = sy.TorchHook(th)
from torch import nn, optim

W0626 14:22:38.417521 4602033600 hook.py:97] Torch was already hooked... skipping hooking process


In [89]:
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
secure_worker = sy.VirtualWorker(hook, id="secure_worker")

In [90]:
bob.add_workers([alice, secure_worker])
alice.add_workers([bob, secure_worker])
secure_worker.add_workers([alice, bob])

W0626 14:22:38.430925 4602033600 base.py:628] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
W0626 14:22:38.432159 4602033600 base.py:628] Worker secure_worker already exists. Replacing old worker which could cause                     unexpected behavior
W0626 14:22:38.432981 4602033600 base.py:628] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior
W0626 14:22:38.433859 4602033600 base.py:628] Worker secure_worker already exists. Replacing old worker which could cause                     unexpected behavior
W0626 14:22:38.434483 4602033600 base.py:628] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
W0626 14:22:38.435142 4602033600 base.py:628] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior


<VirtualWorker id:secure_worker #objects:49>

In [91]:
data = th.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = th.tensor([[0],[0],[1],[1.]], requires_grad=True)

In [92]:
bobs_data = data[0:2].send(bob)
bobs_target = target[0:2].send(bob)

In [93]:
alices_data = data[2:].send(alice)
alices_target = target[2:].send(alice)

In [94]:
model = nn.Linear(2,1)

In [95]:
for round_iter in range(10):

    bobs_model = model.copy().send(bob)
    alices_model = model.copy().send(alice)

    bobs_opt = optim.SGD(params=bobs_model.parameters(), lr=0.1)
    alices_opt = optim.SGD(params=alices_model.parameters(), lr=0.1)

    for i in range(10):

        bobs_opt.zero_grad()
        bobs_pred = bobs_model(bobs_data)
        bobs_loss = ((bobs_pred - bobs_target) **2).sum()
        bobs_loss.backward()

        bobs_opt.step()
        bobs_loss = bobs_loss.get().data
        bobs_loss

        alices_opt.zero_grad()
        alices_pred = alices_model(alices_data)
        alices_loss = ((alices_pred - alices_target) **2).sum()
        alices_loss.backward()

        alices_opt.step()
        alices_loss = alices_loss.get().data
        alices_loss

    alices_model.move(secure_worker)
    bobs_model.move(secure_worker)

    with th.no_grad():
        model.weight.set_(((alices_model.weight.data + bobs_model.weight.data) / 2).get())
        model.bias.set_(((alices_model.bias.data + bobs_model.bias.data) / 2).get())
        
    print("Bob:" + str(bobs_loss) + " Alice:" + str(alices_loss))

Bob:tensor(0.0077) Alice:tensor(0.0129)
Bob:tensor(0.0008) Alice:tensor(0.0047)
Bob:tensor(1.2198e-06) Alice:tensor(0.0014)
Bob:tensor(0.0002) Alice:tensor(0.0005)
Bob:tensor(0.0003) Alice:tensor(0.0002)
Bob:tensor(0.0003) Alice:tensor(9.7346e-05)
Bob:tensor(0.0002) Alice:tensor(5.7544e-05)
Bob:tensor(0.0002) Alice:tensor(3.7786e-05)
Bob:tensor(0.0001) Alice:tensor(2.6412e-05)
Bob:tensor(0.0001) Alice:tensor(1.9115e-05)


# Lesson: Intro to Additive Secret Sharing

While being able to have a trusted third party to perform the aggregation is certainly nice, in an ideal setting we wouldn't have to trust anyone at all. This is where Cryptography can provide an interesting alterantive. 

Specifically, we're going to be looking at a simple protocol for Secure Multi-Party Computation called Additive Secret Sharing. This protocol will allow multiple parties (of size 3 or more) to aggregate their gradients without the use of a trusted 3rd party to perform the aggregation. In other words, we can add 3 numbers together from 3 different people without anyone ever learning the inputs of any other actors.

Let's start by considering the number 5, which we'll put into a varible x

In [96]:
x = 5

Let's say we wanted to SHARE the ownership of this number between two people, Alice and Bob. We could split this number into two shares, 2, and 3, and give one to Alice and one to Bob

In [97]:
bob_x_share = 2
alice_x_share = 3

decrypted_x = bob_x_share + alice_x_share
decrypted_x

5

Note that neither Bob nor Alice know the value of x. They only know the value of their own SHARE of x. Thus, the true value of X is hidden (i.e., encrypted). 

The truly amazing thing, however, is that Alice and Bob can still compute using this value! They can perform arithmetic over the hidden value! Let's say Bob and Alice wanted to multiply this value by 2! If each of them multiplied their respective share by 2, then the hidden number between them is also multiplied! Check it out!

In [98]:
bob_x_share = 2 * 2
alice_x_share = 3 * 2

decrypted_x = bob_x_share + alice_x_share
decrypted_x

10

This even works for addition between two shared values!!

In [99]:
# encrypted "5"
bob_x_share = 2
alice_x_share = 3

# encrypted "7"
bob_y_share = 5
alice_y_share = 2

# encrypted 5 + 7
bob_z_share = bob_x_share + bob_y_share
alice_z_share = alice_x_share + alice_y_share

decrypted_z = bob_z_share + alice_z_share
decrypted_z

12

As you can see, we just added two numbers together while they were still encrypted!!!

One small tweak - notice that since all our numbers are positive, it's possible for each share to reveal a little bit of information about the hidden value, namely, it's always greater than the share. Thus, if Bob has a share "3" then he knows that the encrypted value is at least 3.

This would be quite bad, but can be solved through a simple fix. Decryption happens by summing all the shares together MODULUS some constant. I.e.

In [100]:
x = 5

Q = 23740629843760239486723

bob_x_share = 23552870267 # <- a random number
alice_x_share = Q - bob_x_share + x
alice_x_share

23740629843736686616461

In [101]:
(bob_x_share + alice_x_share) % Q

5

So now, as you can see, both shares are wildly larger than the number being shared, meaning that individual shares no longer leak this inforation. However, all the properties we discussed earlier still hold! (addition, encryption, decryption, etc.)

# Project: Build Methods for Encrypt, Decrypt, and Add 

In this project, you must take the lessons we learned in the last section and write general methods for encrypt, decrypt, and add. Store shares for a variable in a tuple like so.

In [102]:
x_share = (2,5,7)

Even though normally those shares would be distributed amongst several workers, you can store them in ordered tuples like this for now :)

In [103]:
# try this project here!
import random

In [104]:
Q = 237406298764329864985709753492875

In [105]:
def encrypt(x, n_shares=3):

    shares = list()
    
    for i in range(n_shares - 1):
        shares.append(random.randint(0,Q))
    
    final_share = Q - (sum(shares) % Q) + x
    
    shares.append(final_share)
    
    return tuple(shares)

In [106]:
def decrypt(shares):
    return sum(shares) % Q

In [107]:
def add(a, b):
    
    c = list()
    
    assert(len(a) == len(b))
    
    for i in range(len(a)):
        c.append((a[i] + b[i]) % Q)
        
    return tuple(c)

In [108]:
decrypt(add(encrypt(5),encrypt(10)))

15

# Lesson: Intro to Fixed Precision Encoding

As you may remember, our goal is to aggregate gradients using this new Secret Sharing technique. However, the protocol we've just explored in the last section uses positive integers. However, our neural network weights are NOT integers. Instead, our weights are decimals (floating point numbers).

Not a huge deal! We just need to use a fixed precision encoding, which lets us do computation over decimal numbers using integers!

In [109]:
BASE=10
PRECISION=4

In [110]:
def encode(x):
    return int((x * (BASE ** PRECISION)) % Q)

def decode(x):
    return (x if x <= Q/2 else x - Q) / BASE**PRECISION

In [111]:
encode(3.5)

35000

In [112]:
decode(35000)

3.5

In [113]:
x = encrypt(encode(5.5))
y = encrypt(encode(2.3))
z = add(x,y)
decode(decrypt(z))

7.8

# Lesson: Secret Sharing + Fixed Precision in PySyft

While writing things from scratch is certainly educational, PySyft makes a great deal of this much easier for us through its abstractions.

In [114]:
bob = bob.clear_objects()
alice = alice.clear_objects()
secure_worker = secure_worker.clear_objects()

In [115]:
x = th.tensor([1,2,3,4,5])

### Secret Sharing Using PySyft

We can share using the simple .share() method!

In [116]:
x = x.share(bob, alice, secure_worker)

In [117]:
bob._objects

{}

In [118]:
x

(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:90002498453 -> bob:37090641820]
	-> (Wrapper)>[PointerTensor | me:31643138087 -> alice:73114483220]
	-> (Wrapper)>[PointerTensor | me:61548710901 -> secure_worker:4343473864]
	*crypto provider: me*

and as you can see, Bob now has one of the shares of x! Furthermore, we can still call addition in this state, and PySyft will automatically perform the remote execution for us!

In [119]:
y = x + x

KeyError: 'Object "37090641820" not found on worker!!!You just tried to interact with an object ID:37090641820 on <VirtualWorker id:bob #objects:0> which does not exist!!! Use .send() and .get() on all your tensors to make sure they\'reon the same machines. If you think this tensor does exist, check the ._objects dictionaryon the worker and see for yourself!!! The most common reason this error happens is because someone calls.get() on the object\'s pointer without realizing it (which deletes the remote object and sends it to the pointer). Check your code to make sure you haven\'t already called .get() on this pointer!!!'

In [84]:
y

(43610098826456891177803438456028,
 205739860985540133232902371775107,
 225462637716662705560713696777615)

In [85]:
y.get()

AttributeError: 'tuple' object has no attribute 'get'

### Fixed Precision using PySyft

We can also convert a tensor to fixed precision using .fix_precision()

In [37]:
x = th.tensor([0.1,0.2,0.3])

In [38]:
x

tensor([0.1000, 0.2000, 0.3000])

In [39]:
x = x.fix_prec()

In [40]:
x.child.child

tensor([100, 200, 300])

In [41]:
y = x + x

In [42]:
y = y.float_prec()
y

tensor([0.2000, 0.4000, 0.6000])

### Shared Fixed Precision

And of course, we can combine the two!

In [43]:
x = th.tensor([0.1, 0.2, 0.3])

In [44]:
x = x.fix_prec().share(bob, alice, secure_worker)

In [45]:
y = x + x

In [46]:
y.get().float_prec()

tensor([0.2000, 0.4000, 0.6000])

Make sure to make the point that people can see the model averages in the clear.

# Final Project: Federated Learning with Encrypted Gradient Aggregation